In [1]:
import json
from pytket import Circuit
from pytket.circuit.display import render_circuit_jupyter

### Load circuit

In [2]:
with open('test_files/demo_circ.json', 'r') as f:
    data = json.load(f)

circuit = Circuit.from_dict(data)
render_circuit_jupyter(circuit)

### Load precompiled Circuit Rewriter

In [3]:
from tket2.optimiser import BadgerOptimiser
opt = BadgerOptimiser.load_precompiled('matcher.rwr')

### Explore the rewrite space

In [4]:
all_rewrites = opt.run_portdiff(circuit, timeout=10)

In [5]:
all_rewrites.render_jupyter()

### Find the best rewrites using SAT

In [6]:
from tket2.optimiser import construct_z3_optimiser

sat_problem = construct_z3_optimiser(all_rewrites, [])
print(sat_problem)

(declare-fun rw_3 () Bool)
(declare-fun rw_2 () Bool)
(declare-fun rw_1 () Bool)
(declare-fun rw_0 () Bool)
(assert (or rw_0 rw_1 rw_2 rw_3))
(assert (=> rw_1 rw_0))
(assert (not (and rw_1 rw_3)))
(assert (not (and rw_1 rw_3)))
(assert (=> rw_2 rw_0))
(assert (=> rw_2 rw_0))
(assert (not (and rw_2 rw_3)))
(assert (not (and rw_2 rw_3)))
(assert (=> rw_2 rw_0))
(assert (not (and rw_2 rw_3)))
(assert (=> rw_3 rw_0))
(assert (=> rw_3 rw_0))
(assert (=> rw_3 rw_0))
(assert-soft rw_1 :weight 2)
(assert-soft rw_2 :weight 2)
(assert-soft rw_3 :weight 1)
(check-sat)



In [7]:
sat_problem.check()

sat

In [8]:
model = sat_problem.model()
model

[rw_0 = True, rw_1 = True, rw_2 = True, rw_3 = False]

### Extract the optimised circuit

In [9]:
selected = [i for (i, b) in enumerate(model) if model[b]]
opt_circuit = all_rewrites.extract_circuit(selected)

render_circuit_jupyter(opt_circuit.to_tket1())

## Or end-to-end in one compiler pass:

In [10]:
from tket2.passes import badger_pass

p = badger_pass(rewriter="matcher.rwr", timeout=1)

p.apply(circuit)
render_circuit_jupyter(circuit)